# WINDNODE ABW - Multi Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__authors__ 		= "[Guido Pleßmann](https://github.com/gplssm), [Jonathan Amme](https://github.com/nesnoj), [Julian Endres](https://github.com/nailend), " <br>

<div class="alert alert-warning">
<font size="4">We kindly ask for your patience as the rendering process can take some time.</font>


</div>

## Intro

This jupyter notebook provides plots and information to compare the results of the dispatch-optimization of __all__ [scenarios](https://windnode-abw.readthedocs.io/en/dev/scenarios.html) by the study [__"A regional energy system model for Anhalt-Bitterfeld-Wittenberg"__](https://windnode-abw.readthedocs.io/en/dev/). The different scenarios cover various combinations of renewable energy penetration, area restrictions and flexibility options in heat and power sector. The notebooks will, therefore, give an overall view of energy supply and demand by the various scenarios and an insight into scenario-specific distribution and flexiblity effects.
    

  **The representation in jupyter notebooks is intended to ensure transparency and to provide a low entry barrier for further analysis.* 

<div class="alert alert-info">
<b>Notes on plots</b>


<ol>
<li> Some plots are generated with plotly and may not show up initially as Javascript is not enabled by default.</li>
<li> This can be solved by clicking <b>File -> "Trust Notebook"</b>.</li>
</ol> 
    
<b>These plots have interactive features:</b>

<ul>
<li> hoovering over the plot will display additional infos </li>
<li> clicking the legend selects data </li>
</ul>
    
    
</div>

# Table of Contents

* [Scenario information](#scenario_information)
* [0 Key Results](#0_key_results)
* [1 Demand and Generation](#1_demand_and_generation)
* [2 Area required by RES](#2_area_required_by_res)
* [3 Energy Exchange and Autarky](#3_exchange_autarky)
* [4 Energy Mix](#4_energy_mix)
* [5 Emissions](#5_emissions)
* [6 Costs](#6_costs)
* [7 Power Grid](#7_power_grid)
* [8 Flexibility](#8_flexibility)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
} 

In [ ]:
######## WINDNODE ###########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()
# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')
# import scripts
from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######## DATA ###########
import re
import pandas as pd

######## PLOTTING ###########
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
# set seaborn style
sns.set()
# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Scenario information<a class="anchor" id="scenario_information"></a>

TODO
- [ ] suppress long log output of analysis
- [ ] use consistent scenario order, cf. https://github.com/windnode/WindNODE_ABW/issues/64#issuecomment-670712969

In [ ]:
# obtain processed results
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=scenarios,
                                      force_new_results=force_new_results)

### Municipality Names

In [ ]:
# Names of Municialities
MUN_NAMES = regions_scns[scenarios[0]].muns.gen.to_dict()
# extend for total ABW region
MUN_NAMES.update({100:'ABW'})

# 0 Key Results <a class="anchor" id="0_key_results"></a>

The following plots provide an overview on the key results of all scenarios.

TODO
- [X] Scenario order?
- [ ] add explanations on params
- [ ] rework autarky as soon as #135 is merged

In [ ]:
scenario_results = plot_key_scenario_results(results_scns, scenarios, scenario_order=scenario_order)

# 1 Demand and Generation <a class="anchor" id="1_demand_and_generation"></a>

## 1.1 Installed Capacities Electricity/Heat
The following figures show the total installed capacity for the ABW-region for each scenario.

In [ ]:
cap_el = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity electricity supply'].sum(axis=0)
                       for scn in scenarios}).T

cap_el = cap_el.assign(sum=cap_el.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1)

fig = go.Figure()

for tech, data in cap_el.iteritems():
    fig.add_trace(go.Bar(x=cap_el.index,
                         y=data,
                         name=PRINT_NAMES[tech],
                         marker=dict(color=COLORS[tech]),
                         hovertemplate='%{y:.1f} MW',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Installed Capacity Electricity Supply',
               y=1),
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis_tickfont_size=12,
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order),
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True,)

fig.show()

In [ ]:
cap_th = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity heat supply'].sum(axis=0)
                       for scn in scenarios}).T

cap_th = cap_th.assign(sum=cap_th.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1)

fig = go.Figure()
colors_el = [COLORS[c] for c in cap_th.columns]
for tech, data in cap_th.iteritems():
    fig.add_trace(go.Bar(x=cap_th.index,
                         y=data,
                         name=PRINT_NAMES[tech],
                         marker=dict(color=COLORS[tech]),
                         hovertemplate='%{y:.1f} MW',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Installed Capacity Heat Supply',
               y=1),
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order),
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

## 1.2 Electricity and Heat Generation
The following figures show either the annual total of generated heat or heat for the ABW-region for each scenario.

In [ ]:
df_gen_el = pd.DataFrame({scn: results_scns[scn]['results_t']['Electricity generation']
                       for scn in scenarios}).T
df_gen_el = df_gen_el / 1e3 #
fig = go.Figure()
for tech, data in df_gen_el.iteritems():
    fig.add_trace(go.Bar(x=df_gen_el.index,
                         y=data,
                         name=tech,
                         marker=dict(color=COLORS_PRINT[tech]),
                         hovertemplate='%{y:.1f} GWh',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Electricity Generation',
               y=1),
    barmode='stack',
    height=800,
    hovermode="x unified",
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order),
    yaxis=dict(title='GWh',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

In [ ]:
df_gen_heat = pd.DataFrame({scn: results_scns[scn]['results_t']['Heat generation']
                       for scn in scenarios}).T
df_gen_heat = df_gen_heat / 1e3 #
fig = go.Figure()
for tech, data in df_gen_heat.iteritems():
    fig.add_trace(go.Bar(x=df_gen_heat.index,
                         y=data,
                         name=tech,
                         marker=dict(color=COLORS_PRINT[tech]),
                         hovertemplate='%{y:.1f} GWh',
                         showlegend=True))

fig.update_layout(
    title=dict(text='Heat Generation',
               y=1),
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order),
    yaxis=dict(title='GWh',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

## 1.3 Electricity and Heat Demand
The following figure shows both the annual total demand of electricity and heat for the ABW-region for each scenario.

In [ ]:
df_el = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Stromnachfrage nach Gemeinde'].sum()
                       for scn in scenarios}).T
df_heat = pd.DataFrame({scn: results_scns[scn]['results_axlxt']['Wärmenachfrage nach Gemeinde'].sum()
                       for scn in scenarios}).T
df_heat = df_heat.T.groupby([s.split('_')[0] for s in df_heat.columns.values]).sum().T

df_demand = pd.concat([df_el, df_heat], axis=0, keys=['Electricity', 'Heat'], sort=False)
df_demand = df_demand.rename(columns=PRINT_NAMES)
df_demand = df_demand / 1e3

fig = make_subplots(rows=len(scenarios), cols=1, shared_xaxes=True, vertical_spacing=0)
for row, (key, df) in enumerate(df_demand.groupby(level=1)):
    for i, (tech, data) in enumerate(df.iteritems()):
        fig.add_trace(go.Bar(y=list(zip(*data.index.swaplevel())),
                             x=data,
                             name=tech,
                             orientation='h',
                             marker_color=COLORS_PRINT[tech],
                             legendgroup=tech,
                             hovertemplate='%{x:.1f} GWh',
                             showlegend=not bool(row)), row=row+1, col=1)

fig.update_layout(
    title='Demand',
    barmode='stack',
    hovermode="y unified",
    yaxis=dict(tickfont_size=12,
              categoryorder='array',
              categoryarray=scenario_order),
    height=50*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_xaxes(title_text="GWH", row=len(scenarios), col=1, matches='x')
fig.show() 

# 2 Area required by RES <a class="anchor" id="2_area_required_by_res"></a>

## 2.1 Available Areas: Installable Power

The following plot shows the maximum installable power for each land use scenario and technology.

Absolute values are represented in <b>colored</b> bars and relative (always relative to the max. installable power for the regulatory status quo) in  <b>grey</b>. The regulatory status quo is highlighted in <b>bold</b> for each technology.

<div class="alert alert-block alert-info">

<ul>
<li> Technology-specific naming conventions of land use scenarios:
    <ul>
        <li><b>Wind</b>: Distance to settlements <i>(500m/1000m)</i> |  use of forests <i>(with: w / without: wo)</i> | percentage of available area due to restrictions resulting from case-by-case decisions</li>
        <li> <b>PV ground</b>: Restrictions that apply (hard: H / hard+soft: HS)| percentage of total available agricultural area</li>
    </ul></li>
    </ul>
    
    
Note: The specific land use (ha/MWp) of PV are different for 2035 and 2050. For the sake of simplicity, the value of 2050 is used in this plot (cf. assumptions).

</div>

In [ ]:
power_pot_land_use(regions_scns, scenarios)

TODO
- [ ] conversions info

The following plot shows the maximum installable power for each RES scenario.

<div class="alert alert-block alert-info">

 RES scenario convertion:
    <ul>
        <li><b>RE- </b>: </li>
        <li><b>RE </b>: </li>
        <li><b>RE+ </b>: </li>
        <li><b>RE++ </b>: </li>
    </ul>

and technology as absolute values (colored) and relative to the max. installable power for the regulatory status quo (red).

In [ ]:
power_pot_scenarios(regions_scns, scenarios)

### 2.1.1 Available areas: Wind Energy

- [ ] normalize colors?
- [ ] 

In [ ]:
df_pot_area = pd.DataFrame({scn: area_ha.droplevel(level=1)
              for scn, area_ha
              in regions_scns['StatusQuo'].pot_areas_wec.area_ha.groupby(level=1)})

df_pot_area.drop(columns='s1000f0', inplace=True)

name_mapping = {'sq': 'Wind legal SQ', 's1000f1': 'Wind 100m w forest 10-perc', 's500f0': 'Wind 500m wo forest 10-perc', 's500f1': 'Wind 500m w forest 10-perc'}

df_pot_area.rename(columns=name_mapping, inplace=True)

fig, axes = plt.subplots(2, 2, figsize=(12,6))

for ax, (key, data) in  zip(axes.flat, df_pot_area.iteritems()):
    plot_geoplot(key, data, regions_scns['StatusQuo'], ax=ax, unit='ha')
    
fig.suptitle('Required Area',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

### 2.1.2 Available areas: Ground-Mounted PV

- [ ] normalize colors?
- [ ] Add note that the shown agri areas are 100%, we use 0.1% and 1% in our scenarios.
- [ ] A comprehensive description of how those areas are determined see StEmp ABW docs.


In [ ]:
df_pot_area = pd.DataFrame({scn: area_ha.droplevel(level=1)
              for scn, area_ha
              in regions_scns['StatusQuo'].pot_areas_pv.area_ha.groupby(level=1)})

#df_pot_area.drop(columns='s1000f0', inplace=True)

#name_mapping = {'sq': 'Wind legal SQ', 's1000f1': 'Wind 100m w forest 10-perc', 's500f0': 'Wind 500m wo forest 10-perc', 's500f1': 'Wind 500m w forest 10-perc'}

#df_pot_area.rename(columns=name_mapping, inplace=True)

fig, axes = plt.subplots(3, 2, figsize=(12,10))

for ax, (key, data) in  zip(axes.flat, df_pot_area.iteritems()):
    plot_geoplot(key, data, regions_scns['StatusQuo'], ax=ax, unit='ha')
    
fig.suptitle('Available Areas: Ground-Mounted',
     fontsize=16,
     fontweight='normal',
            y=1)
plt.tight_layout()
plt.show()

### 2.1.3 Available areas: Roof-Mounted PV

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plot_geoplot('PV Roof-Mounted',
             regions_scns['StatusQuo'].pot_areas_pv_roof.sum(axis=1).rename('area'),
             regions_scns['StatusQuo'], ax=ax, unit='ha')

# 3 Energy Exchange and Autarky <a class="anchor" id="3_exchange_autarky"></a>

## 3.1 Energy Exchange

## 3.2 Autarky

- [ ] add equations relevant for ABW
- [ ] explain what sectors is included in "autarky" (el or el+th?)

# 4 Energy Mix <a class="anchor" id="4_energy_mix"></a>

### 4.1 Electricity Generation and Demand

In [ ]:
df_generation = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Stromerzeugung'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)
df_demand = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Stromnachfrage'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

fig = go.Figure()
for tech, data in df_generation.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=data / 1e3,
                            orientation='v',
                            name=tech,
#                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh'))

for tech, data in df_demand.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=-data / 1e3,
                            orientation='v',
                            name=tech,
#                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh',
#                             visible='legendonly'
                        ))


fig.update_layout(
    title='Power Generation and Demand',
    barmode='relative',
#     height=600,
    autosize=True,
    hovermode="x unified")
fig.update_yaxes(title='GWh', showspikes=False)
fig.show()

## 4.2 Heat Generation and Demand

In [ ]:
df_generation = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Wärmeerzeugung'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)
df_demand = pd.DataFrame({scn: results_scns[scn]['flows_txaxt']['Wärmenachfrage'].sum()
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

fig = go.Figure()
for tech, data in df_generation.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=data / 1e3,
                            orientation='v',
                            name=tech,
#                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh'))

for tech, data in df_demand.iteritems():
    fig.add_trace(go.Bar(x=data.index,
                            y=-data / 1e3,
                            orientation='v',
                            name=tech,
#                             marker_color=COLORS[tech],
                            hovertemplate='%{y:.1f} GWh',
#                             visible='legendonly'
                        ))


fig.update_layout(
    title='Heat Generation and Demand',
    barmode='relative',
#     height=600,
    autosize=True,
    hovermode="x unified")
fig.update_yaxes(title='GWh', showspikes=False)
fig.show()

# 5 Emissions <a class="anchor" id="5_emissions"></a>

## 5.1 Emissions absolute

In [ ]:
df_emissions = pd.DataFrame(
    {scn : pd.concat([pd.Series(data=results_scns[scn]['results_axlxt'][key].sum().sum(), index=[key]) \
                      for key in results_scns[scn]['results_axlxt'] if 'total' in key]) \
     for scn in scenarios}).T

df_emissions['CO2 emissions grid total'] = df_emissions[['CO2 emissions grid total',
                                                         'CO2 emissions grid new total']]
df_emissions = df_emissions.drop(columns='CO2 emissions grid new total')

fig = go.Figure()
for i, (tech, data) in enumerate(df_emissions.iteritems()):
    fig.add_trace(go.Bar(x=data.index,
                         y=data,
                         name=tech,
                         orientation='v',
                         #marker_color=CMAP[i+1],
                         legendgroup=tech,
                         hovertemplate='%{y:.1f} t CO2',))
fig.update_layout(
    title='Emissions',
    barmode='stack',
    hovermode="x unified",
    #height=50*len(scenarios),
    legend= {'tracegroupgap': 0},
    autosize=True)

fig.update_yaxes(title_text="t CO2")
fig.show() 

# 6 Costs <a class="anchor" id="6_costs"></a>

## 6.1 Costs per Sector and Technology

## 6.2 LCOE, LCOH and Renewable Share

In [ ]:
scenario_results = scenario_results[1].join(scenario_results[2].drop("Scenario", axis=1)).join(scenario_results[3].drop("Scenario", axis=1))

tmp_dict = {
    "RES share [%]": [],
    "Autark hours [%]": [],
    'Heat Storage Use [GWh]': [],
    'El. Storage Use [GWh]': [],
    'Net DSM activation [GWh]': []
}
for scenario in scenarios:
    tmp_dict["RES share [%]"].append(results_scns[scenario]["results_t"]['Electricity generation'].drop(
        ["Large-scale CHP", "Open-cycle gas turbine", "Combined-cycle gas turbine"]).sum() / float(results_scns[scenario]["highlevel_results"]["Electricity demand total"]) * 100)
    tmp_dict["Autark hours [%]"].append(float(results_scns[scenario]["highlevel_results"]["Autark hours"]) * 100)
    tmp_dict['Heat Storage Use [GWh]'].append(results_scns[scenario]["results_axlxt"]['Wärmespeicher nach Gemeinde'].sum().sum() / 1e3)
    tmp_dict['El. Storage Use [GWh]'].append(results_scns[scenario]["results_axlxt"]['Batteriespeicher nach Gemeinde'].sum().sum() / 1e3)
    tmp_dict['Net DSM activation [GWh]'].append(float(results_scns[scenario]["highlevel_results"]['Net DSM activation']) / 1e3)
additional_key_figures = pd.DataFrame(tmp_dict, index=scenarios)
scenario_results_ext = scenario_results.join(additional_key_figures, on="Scenario").set_index("Scenario").sort_values("RES share [%]")
scenario_results_ext = scenario_results_ext.drop("StatusQuo", axis=0)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

re_scn_matches = [re.search('((RE\+\+)|(RE\+)|(WIND\+)|(PV\+)|(RE-)|(RE))', scn) for scn in scenario_results_ext.index]
scenario_results_ext["RE scenario"] = [m[0].replace("WIND", "RE").replace("PV", "RE") if m is not None else "RE" for m in re_scn_matches]

re_scn_sort = {"RE-": 0, "RE": 1, "RE+": 2, "RE++": 3}
scenario_results_ext["RE scenario order"] = scenario_results_ext["RE scenario"].replace(re_scn_sort)
scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "RES share [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["RES share [%]"],
            x=xlabels,
            name="RES share in %",
            #marker_color=CMAP[4],
            #hovertemplate='%{y:.2f}<extra></extra> %',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext["LCOE [EUR/MWh]"],
                             x=xlabels,
                             mode='markers',
                             opacity=0.7,
                             #marker_color=CMAP[0],
                             name="LCOE in EUR/MWh",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext["LCOH [EUR/MWh]"],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="indianred",
                             name="LCOH in EUR/MWh",
                             ), secondary_y=True)


fig.update_layout(yaxis=dict(
                      title="RES share in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="LCOE/LCOH in EUR/MWh", secondary_y=True)
fig.update_xaxes()

# 7 Power Grid <a class="anchor" id="7_power_grid"></a>

# 8 Flexibility <a class="anchor" id="8_flexibility"></a>

In [ ]:
scenario_results_ext["Flexibility shifted energy [GWh]"] = scenario_results_ext[["Heat Storage Use [GWh]", "El. Storage Use [GWh]","Net DSM activation [GWh]"]].sum(axis=1)

re_color_mapping = {"RE-": "mediumaquamarine", "RE": "lightseagreen", "RE+": "seagreen", "RE++": "yellowgreen"}
colors = [re_color_mapping[_] for _ in scenario_results_ext["RE scenario"]]

pairs = {
    "LCOE vs. Exports": ("LCOE [EUR/MWh]", "El. Exports [TWh]",),
    "Flexibility vs. Exports": ("Flexibility shifted energy [GWh]", "El. Exports [TWh]"),
    "Flexibility vs. Autarky": ("Flexibility shifted energy [GWh]", "Autarky [%]"),
    "LCOE vs. Autarky": ("LCOE [EUR/MWh]", "Autarky [%]",),
    "Flexibility vs. Intra-reg. Exchange": ("Flexibility shifted energy [GWh]", "Intra-reg. Exchange [TWh]"),
    "Autarky vs. Intra-reg. Exchange": ("Autarky [%]", "Intra-reg. Exchange [TWh]"),
    "Autark hours vs. Intra-reg. Exchange": ("Autark hours [%]", "Intra-reg. Exchange [TWh]"),
    "Flexibility vs. Emissions": ("Flexibility shifted energy [GWh]", "Specific Emissions [g/kWh]"),
}

for k, pair in pairs.items():
    fig = go.Figure(data=go.Scatter(
        x=scenario_results_ext[pair[1]],
        y=scenario_results_ext[pair[0]],
        mode='markers',
        marker=dict(size=[20] * len(scenarios),
                    color=colors),
        text=scenario_results_ext.index
    ))


    fig.update_yaxes(title_text=pair[0])
    fig.update_xaxes(title_text=pair[1])
    fig.show()

### 8.0.4 Flexibility utilization vs. Autarky

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "Autark hours [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["Autark hours [%]"],
            x=xlabels,
            name="Autark hours in %",
           # marker_color=CMAP[4],
            #hovertemplate='%{y:.2f}<extra></extra> %',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext['El. Storage Use [%]'],
                             x=xlabels,
                             mode='markers',
                             opacity=0.7,
                             #marker_color=CMAP[0],
                             name="Battery storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Heat Storage Use [%]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="indianred",
                             name="Heat storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['DSM Utilization Rate [%]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="darkorange",
                             name="DSM utilization in %",
                             ), secondary_y=True)


fig.update_layout(yaxis=dict(
                      title="Autark hours in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="Utilization in %", secondary_y=True)
fig.update_xaxes()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "Autark hours [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["Autark hours [%]"],
            x=xlabels,
            name="Autark hours in %",
            #marker_color=CMAP[4],
            hovertemplate='%{y:.2f}<extra></extra> %',
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext['El. Storage Use [GWh]'],
                             x=xlabels,
                             mode='markers',
                             opacity=0.7,
                             #marker_color=CMAP[0],
                             name="Shifted electricity battery storage in GWh",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Heat Storage Use [GWh]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="indianred",
                             name="Shifted electricity heat storage in GWh",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Net DSM activation [GWh]'],
                             x=xlabels,
                             mode='markers',
                             opacity=0.7,
                             marker_color="darkorange",
                             name="Shifted electricity DSM in GWh",
                             ), secondary_y=True)


fig.update_layout(yaxis=dict(
                      title="Autark hours in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="Shifted electricity in GWh", secondary_y=True)
fig.update_xaxes()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

scenario_results_ext = scenario_results_ext.sort_values(["RE scenario order", "RES share [%]"])
xlabels = [["<b>{}</b>".format(_) for _ in scenario_results_ext["RE scenario"]], scenario_results_ext.index]

fig.add_trace(go.Bar(y=scenario_results_ext["RES share [%]"],
            x=xlabels,
            name="RES share in %",
            #marker_color=CMAP[4],
            showlegend=True))

fig.add_trace(go.Scatter(y=scenario_results_ext['El. Storage Use [%]'],
                             x=xlabels,
                             mode='markers',
                             opacity=0.7,
                             #marker_color=CMAP[0],
                             name="Battery storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['Heat Storage Use [%]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="indianred",
                             name="Heat storage utilization in %",
                             ), secondary_y=True)

fig.add_trace(go.Scatter(y=scenario_results_ext['DSM Utilization Rate [%]'],
                             x=xlabels,
                            mode='markers',
                             opacity=0.7,
                             marker_color="darkorange",
                             name="DSM utilization in %",
                             ), secondary_y=True)


fig.update_layout(yaxis=dict(
                      title="RES share in %",
                      titlefont_size=16,
                      tickfont_size=12),
                  height=700,
                  autosize=True,
                  hovermode="x unified",
                  legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right",x=1)
                 )
fig.update_yaxes(title_text="Utilization in %", secondary_y=True)
fig.update_xaxes()